# Sound Field Synthesis

[return to main page](index.ipynb)

## Preparations

In [2]:
import tools
import numpy as np
import sounddevice as sd  # for playback
import soundfile as sf  # for reading a soundfile
from scipy import signal

# remove "inline" to get a separate plotting window:
%matplotlib inline
import matplotlib.pyplot as plt

## Basic Principle

<img src="data/sfs/sfs_discrete.png" width="400"/>

The goal is to synthesize the sound field of a virtual source $S(\mathbf{x},\omega)$ within the listening area (yellow) with an ensemble of individually driven secondary sources (loudspeakers) surrounding the area. The reproduced sound field given as

$$
P(\mathbf{x},\omega) = \sum_{l=1}^{L} D(\mathbf{x}_l,\omega) \; G(\mathbf{x}|\mathbf{x}_l,\omega)
$$

which states the superposition of the sound fields $G(\mathbf{x}|\mathbf{x}_l,\omega)$ emitted by each individual secondary sources. Here, each loudspeaker emits its respective **driving signal** $D(\mathbf{x}_l,\omega)$. 

<img src="data/sfs/sfs_continuous.png" width="400"/>

For the continuous case, i.e. the spacing between the secondary sources is zero, the sum migrates to an integral

$$
P(\mathbf{x},\omega)=\oint_{\partial V} D(\mathbf{x}_0,\omega) \; G(\mathbf{x} | \mathbf{x}_0,\omega) \; \mathrm{d}A(\mathbf{x}_0)
$$

which is the so-called **Single Layer Potential**. It needs to be solved for $D(\mathbf{x}_0,\omega)$ under the condition $P(\mathbf{x},\omega) = S(\mathbf{x},\omega)$. Most approaches assume that the all secondary source emit the same sound field $G(\mathbf{x}|\mathbf{x}_0,\omega) = G(\mathbf{x} - \mathbf{x}_0,\omega)$ and free-field propagation. The loudspeakers sound field is modelled as a point source by the free-field Green's function $G_0$

$$
P(\mathbf{x},\omega)=\oint_{\partial V} D(\mathbf{x}_0,\omega) \; G_0(\mathbf{x} - \mathbf{x}_0,\omega) \; \mathrm{d}A(\mathbf{x}_0)
$$

Under the assumptions made in the prior section, the *Single Layer Potential* describes a convolution along surface $\partial V$. In the following, we get to know some approaches for solving it.

## Explicit Solution

### Analogies to the Time-Domain

Let's make a small step back and see how we conveniently solve a similar problem in a different domain, namely the time-domain: We assume, that we have measured the output signal $y(t)$ of a LTI-System with unknown impulse response $h(t)$ given the input signal $x(t)$. The three entities are connected over the convolution integral

$$
y(t) = \int_{-\infty}^{\infty} x(\tau) h(t-\tau) \mathrm d \tau\,.
$$

In order to determine $h(t)$ we have to solve the integral with respect to $h(t)$. We can express the signals as the inverse Fourier transform using its respective Fourier spectrum, see

$$
y(t) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} Y(j\omega) e^{+j\omega t} \mathrm d \omega \\
x(\tau) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} X(j\omega) e^{+j\omega \tau} \mathrm d \omega
\\
h(t-\tau) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} H(j\omega) e^{+j\omega t} e^{-j\omega \tau} \mathrm d \omega
$$

We expanded each signal as a superposition of weigthed basis functions $\Psi(\omega,t)$. The weights are the Fourier Spectra, e.g. $Y(j\omega)$, and the basis functions $\Psi(\omega,t)$ are the expontentials, e.g. $e^{+j\omega t}$.
A very important property of this basis functions $e^{+j\omega t}$ is its orthogonality

$$
\frac{1}{2 \pi} \int_{-\infty}^{\infty} e^{+j\omega_1 t} e^{-j\omega_2 t} \mathrm d t = 
\delta(\omega_1 - \omega_2)
$$

with respect to the integral $\int_{-\infty}^{\infty} \mathrm d t$. We insert the expansion into the convolution integral:

$$
\frac{1}{2 \pi} \int_{-\infty}^{\infty} Y(j\omega) e^{+j\omega t} \mathrm d \omega = 
\color{red}{\int_{-\infty}^{\infty}}
    \Big[ 
    \frac{1}{2 \pi} \int_{-\infty}^{\infty} X(j\omega_1) \color{red}{e^{+j\omega_1 \tau}} \mathrm d \omega_1
    \frac{1}{2 \pi} \int_{-\infty}^{\infty} H(j\omega_2) e^{+j\omega_2 t} \color{red}{e^{-j\omega_2 \tau}} \mathrm d \omega_2
    \Big]
\color{red}{\mathrm d \tau}
$$

We can now solve the integral with respect to $\tau$ (red) using the orthogonality of the exponentials. This results in

$$
\frac{1}{2 \pi} \int_{-\infty}^{\infty} Y(j\omega) e^{+j\omega t} \mathrm d \omega = 
    \frac{1}{2 \pi} 
    \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} X(j\omega_1) 
     H(j\omega_2) e^{+j\omega_2 t} \delta(\omega_1 - \omega_2) \, \mathrm d \omega_1 \mathrm d \omega_2
$$

The integral with respect to $\omega_1$ can be solved by the identity for the dirac delta distribution

$$
\int_{-\infty}^{\infty} F(\omega_1) \delta(\omega_1 - \omega_2) \, \mathrm d \omega_1 =  F(\omega_2)
$$

The result reads

$$
\frac{1}{2 \pi} \int_{-\infty}^{\infty} \color{blue}{Y(j\omega)} e^{+j\omega t} \mathrm d \omega = 
\frac{1}{2 \pi}  \int_{-\infty}^{\infty} \color{blue}{X(j\omega_2) H(j\omega_2)} e^{+j\omega_2 t} \mathrm d \omega_2 \,.
$$

Besides the integration variables, the integrals are the same, so the terms within the integrals are equal. We can hence infer, that a convolution of two signals in the time-domain results in a multiplication of the respective spectra in the frequency-domain. Hence, we can get the spectrum of the impulse response via

$$
H(j\omega) = \frac{Y(j\omega)}{X(j\omega)}
$$

and the impulse response via the inverse Fourier transform

$$
h(t) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} \frac{Y(j\omega)}{X(j\omega)} e^{+j\omega t}
$$

What have we learned? First of all, we have to know the basis function, with are orthogonal 
with respect to the integral defined by the convolution. We can solve the convolution integral with respect to $h(t)$ expanding the entities into this basic function, and divide the resulting spectra of $x(t)$ and $y(t)$. $h(t)$ is then given by the inverse transform.

### Solving the Single Layer Potential

Let's get back to the original Problem:

$$
P(\mathbf{x},\omega)=\oint_{\partial V} D(\mathbf{x}_0,\omega) \; G_0(\mathbf{x} - \mathbf{x}_0,\omega) \; \mathrm{d}A(\mathbf{x}_0)
$$

Obviously the convolution is now defined with regard to space along a 2-dimensional surface. As this surface is closed, the functions defined on it have to be periodic: If one moves around the surface back to the original starting point, the functions have to yield the same values. We know from the time-domain that periodic signals are expressed into basis functions using the Fourier series. Contrary to the Fourier Transform, the Fourier series is only defined for frequencies being an integer multiple $n$ of the fundamental frequency $f_0 = \frac{1}{T}$, where $T$ is the period of the signal. The spatial spectra of function on the surface are hence discrete and the frequency variable is an integer, e.g. $n$.

Assuming that we know the basis functions $\Psi_n(\mathbf{x}_0)$, which are orthogonal on the surface, i.e.

$$
\oint_{\partial V} \Psi_{n_2}(\mathbf{x}_0) \Psi^*_{n_1}(\mathbf{x}_0) \; \mathrm{d}A(\mathbf{x}_0) = \delta[n_1 - n_2]\,,
$$

we can expand the involved function over the basis functions

$$
P(\mathbf{x},\omega)= \sum_{n=0}^{\infty} P_n(\omega) \Psi_n(\mathbf{x})\\
D(\mathbf{x},\omega)= \sum_{n=0}^{\infty} D_n(\omega) \Psi_n(\mathbf{x})\\
G(\mathbf{x},\omega)= \sum_{n=0}^{\infty} G_n(\omega) \Psi_n(\mathbf{x})
$$

with $P_n$, $D_n$, and $G_n$ being the respective spatial spectra. The spatial spectrum of the driving is computed analogue to the time-domain example via
$$
D_n(\omega) = \frac{P_n(\omega)}{G_n(\omega)}\,.
$$

The final driving signal is given as
$$
D(\mathbf{x},\omega) = \sum_{n=0}^{\infty} \frac{P_n(\omega)}{G_n(\omega)} \Psi_n(\mathbf{x})\,.
$$

In general, finding the basis function for a specific surface shape is challenging, if not impossible. The explicit solution of the Single Layer Potential is hence only feasible for simple geometries such as spheres, circles, or planes.

### Circular Geometries

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>